# 번역의 어휘나 분위기를 조절하려면 Fine-Tuning 없이도 적용 가능한 여러 기법을 활용할 수 있음
1. 입력텍스트에 페르소나 힌트 추가
2. 디코더 파라미터 추가 < 큰차이는 없을 수 있음>
   1. num_beams=5,           # 더 정확한 후보 탐색
   2. temperature=0.7,       # 보수적 어조
   3. repetition_penalty=1.2, # 반복 단어 감소
4. 후처리 (Post-processing): 규칙기반 어휘 처리
5. 하이브리드 파이프라인 구축
   1. Translate 로 기본 번역 수행
   2. 별도의 분류 모델로 페르소나 판단 (예: Formal/ Casual 분류)
   3. 판단한 결과에 따라 후처리 

In [20]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M",cache_dir='translate/')
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M",cache_dir='translate/')

def translate(input_text):
    inputs = tokenizer(input_text, return_tensors="pt")
    translated_tokens = model.generate(
        **inputs,
        do_sample = True,        
        num_beams=5,           # 더 정확한 후보 탐색
        temperature=0.7,       # 보수적 어조
        repetition_penalty=1.2, # 반복 단어 감소
        max_length=100,
        # 언어 코드 ID 변환
        forced_bos_token_id=tokenizer.convert_tokens_to_ids("kor_Hang"),  # 한국어 타겟 설정, deu_Latn, eng_Latn
    )
    # 번역된 텍스트 디코딩
    return tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]


In [9]:
input_text = "[Formal], how are you?" # [Formal], [Casual], [Slang], [Business] 등 힌트
print(translate(input_text))  # "안녕하세요, 잘 지내시나요?"

[공식], 안녕하세요?


In [11]:
input_text = "[Formal], how are you?" # [Formal], [Casual], [Slang], [Business] 등 힌트
print(translate(input_text))  # "안녕하세요, 잘 지내시나요?"

[공식], 안녕하세요?


In [60]:
import copy
# 더 확장된 버전 (추가 규칙 필요시)
def formalize_korean(text):
    replacements = {
        "있어": "있당께요",
        "있었어": "있었습니다",
        "좀": "조금",          # 구어체 → 격식체
        "걍": "그냥",          # 슬랭 대체
        "~요": "~니다",      # 어미 변환(예: "가요" → "갑니다")
        "네요": "습니다"       # "합니다네요" → "합니다습니다" 방지를 위해 순서 조정 필요
    }
    result = copy.deepcopy(text)
    for informal, formal in replacements.items():        
        result = result.replace(informal, formal)
        
    return result

In [ ]:
input_text = "Henry, i have some problem that i must have to go there" # [Formal], [Casual], [Slang], [Business] 등 힌트
translated = translate(input_text)
formal_translated = formalize_korean(translated)
print(translated)
print(formal_translated)

헨리, 난 거기 가야 할 문제가 있어
헨리, 난 거기 가야 할 문제가 있당께요
